In [1]:
# Fannibal Holiday Card Exchange Matching Tool
# Create Date: 2023-10-19
# Last Update: 2023-11-19

# In this yearly event, participants can sign up to be matched as many time as they specified.
# They will provide the country they are in, and their willingness to be matched with overseas participants.
# This script will do the following:
# (1) Read in the participants info that stored in a csv file.
# (2) Group participants by their countries.
# (3) Match participants who want to be matched locally.
# (4) Match participants who are open to overseas participants.
# (5) Export matching results by applicants as individual text files.

# Future planning:
# 1. Make it more random, so that sender not necessarily getting cards from their receiver list.
# 2. The exported text files are messages to be sent out to participants. In the future, emailing could be automated from here directly.

import csv
import random
import re

In [2]:
# Read in participants info

applicant_info = []

with open('CardExApp.csv', 'r', encoding='utf-8') as signup:    # Besure to use utf-8 encoding
    reader = csv.DictReader(signup)
    for row in reader:
        applicant_info.append({
            "name": row["Name"],
            "email": row["Email"],
            "handle": row["Handle"],
            "address": row["Address"],
            "matches_wanted": int(row["Cards"]),
            "country": row["Country"],
            "overseas": int(row["Overseas"])
        })

In [3]:
# Main matching

# Sort the applicants by group and within the group, sort by name
applicant_info = sorted(applicant_info, key=lambda x: (x["country"], x["name"]))    # In next update, test results without sorting names

# Create a dictionary to store the original order of applicants
applicant_order = {applicant["name"]: i for i, applicant in enumerate(applicant_info)}

# Initialize a dictionary to track the number of matches each applicant has played
matches_played = {applicant["name"]: 0 for applicant in applicant_info}

# Initialize an empty dictionary to store the matches by applicant
matches_by_applicant = {applicant["name"]: [] for applicant in applicant_info}

# Define the minimum matches required
minimum_matches_required = {}
for applicant in applicant_info:
    if applicant["matches_wanted"] <= 5:
        minimum_matches_required[applicant["name"]] = applicant["matches_wanted"]
    elif 6 <= applicant["matches_wanted"] <= 14:
        minimum_matches_required[applicant["name"]] = int(0.7 * applicant["matches_wanted"])
    else:
        minimum_matches_required[applicant["name"]] = int(0.6 * applicant["matches_wanted"])

# Shuffle the applicant_info list to randomize the order within the same group
grouped_applicants = []
current_country = None

for applicant in applicant_info:
    if current_country is None or current_country != applicant["country"]:
        current_group = applicant["country"]
        applicants_in_country = [applicant]
        grouped_applicants.append(applicants_in_country)
    else:
        applicants_in_country.append(applicant)

for applicants_in_country in grouped_applicants:
    random.shuffle(applicants_in_country)

# Generate matches within the same group
for i in range(len(applicant_info)):
    for j in range(i + 1, len(applicant_info)):
        applicant1 = applicant_info[i]
        applicant2 = applicant_info[j]

        if (
            applicant1["country"] == applicant2["country"]
            and matches_played[applicant1["name"]] < minimum_matches_required[applicant1["name"]]
            and matches_played[applicant2["name"]] < minimum_matches_required[applicant2["name"]]
        ):
            match = (applicant1["name"], applicant2["name"])
            matches_by_applicant[applicant1["name"]].append(applicant2["name"])
            matches_by_applicant[applicant2["name"]].append(applicant1["name"])
            matches_played[applicant1["name"]] += 1
            matches_played[applicant2["name"]] += 1

# Shuffle the applicant_info list to randomize the order
random.shuffle(applicant_info)

# Generate matches for those who are open to match with any group
for i in range(len(applicant_info)):
    for j in range(i + 1, len(applicant_info)):
        applicant1 = applicant_info[i]
        applicant2 = applicant_info[j]

        if (
            applicant1["country"] != applicant2["country"]
            and (
                applicant1["overseas"] == 1
                and applicant2["overseas"] == 1
            )
            and matches_played[applicant1["name"]] < minimum_matches_required[applicant1["name"]]
            and matches_played[applicant2["name"]] < minimum_matches_required[applicant2["name"]]
        ):
            match = (applicant1["name"], applicant2["name"])
            matches_by_applicant[applicant1["name"]].append(applicant2["name"])
            matches_by_applicant[applicant2["name"]].append(applicant1["name"])
            matches_played[applicant1["name"]] += 1
            matches_played[applicant2["name"]] += 1

In [4]:
# Function to find the address from applicant_info
def find_address(name):
    for entry in applicant_info:
        if entry["name"] == name:
            return entry["address"], entry["country"]
    return None, None # Break function if the name is not found

# Print the matches grouped by applicant
# for applicant in applicant_info:
#     name = applicant["name"]
#     receivers = matches_by_applicant[name]

# Function to retrieve the number of cards an applicant signed up originally
def cards_wanted(name):
    for entry in applicant_info:
        if entry["name"] == name:
            return entry["matches_wanted"]
        
# Function to retrieve email address of an applicant
def emailto(name):
    for entry in applicant_info:
        if entry["name"] == name:
            return entry["email"]

In [5]:
# Create individual text files for each applicant's matched list
# Files named by senders' normalized full name
for name, receivers in matches_by_applicant.items():
    cleaned_name = re.sub(r'[^a-zA-Z\s]', '', name)
    cleaned_name = cleaned_name.replace(' ', '_')
    file_name = f'matched_lists/{cleaned_name}_list.txt'
    first_name = name.split()[0]

    # Write main message
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(f"{emailto(name)}\n\n")
        file.write(f"Hello {first_name}!\n\nWe\'re happy to announce that you\'ve been matched up with the Fannibals below for the 2023 Holiday Card Exchange. We recommend that you have your cards in the mail no later than December 6. However, deadlines can differ from country to country, so please check with your local postal service provider.\n\n")
        if len(receivers) < cards_wanted(name):
            file.write(f"You\'ll also notice that you were given fewer names than you had requested. Unfortunately, this was the maximum number we could pair you up with, considering the names and numbers that we were given. Hope this doesn\'t take away from the experience for you.\n\n")
        file.write("Happy Holidays!\n\nSunni & Deeker\n\n\n")
        file.write("= * = LIST STARTS = * =\n\n")
        
        # Write matched receivers' information
        for receiver in receivers:
            address, country = find_address(receiver)
            file.write(f"{receiver}\n")
            file.write(f"{address}\n")
            file.write(f"{country}\n\n")
        
        file.write("= * = END OF LIST = * =\n\n")
    # print(f"Matched list for {name} has been saved as {file_name}")

In [6]:
# Export receiver list as a CSV.

with open('matches.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["Sender", "Matches", "Receiver"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for applicant in applicant_info:
        name = applicant["name"]
        receiver = matches_by_applicant[name]
        receiver_list = ', '.join(sorted(receiver, key=lambda x: applicant_order[x]))
        writer.writerow({
            "Sender": name,
            "Matches": len(receiver),
            "Receiver": receiver_list
        })